# TCN Training Only (Clean)

This notebook is for **training only**.
It uses isolated `train_*` variables and a Sharpe-based checkpoint policy.

## 1) Connect to Colab VM and Sync Repo
Run this first.

In [1]:
# Fresh-start cleanup cell (run before importing project modules)
import gc
import shutil
import subprocess
import sys
from pathlib import Path

TRAIN_REPO_URL = "https://github.com/Dave-DKings/tape_tcn_project.git"
TRAIN_REPO_DIR = Path("/content/adaptive_portfolio_rl")

# 1) Sync repo to latest main
if not (TRAIN_REPO_DIR / ".git").exists():
    subprocess.run(["git", "clone", TRAIN_REPO_URL, str(TRAIN_REPO_DIR)], check=True)

subprocess.run(["git", "-C", str(TRAIN_REPO_DIR), "fetch", "origin"], check=True)
subprocess.run(["git", "-C", str(TRAIN_REPO_DIR), "reset", "--hard", "origin/main"], check=True)

# 2) Remove old experiment outputs/checkpoints/cached data
purge_paths = [
    TRAIN_REPO_DIR / "tcn_fusion_results",
    TRAIN_REPO_DIR / "tcn_results",
    TRAIN_REPO_DIR / "tcn_att_results",
    TRAIN_REPO_DIR / "output_logs",
    TRAIN_REPO_DIR / "data" / "phase1_preparation_artifacts",
    TRAIN_REPO_DIR / "data" / "master_features_NORMALIZED.csv",
    TRAIN_REPO_DIR / "data" / "daily_ohlcv_assets.csv",              # forces fresh OHLCV download
    TRAIN_REPO_DIR / "data" / "processed_daily_macro_features.csv",   # forces fresh macro cache build
]

deleted = []
for p in purge_paths:
    if p.is_dir():
        shutil.rmtree(p, ignore_errors=True)
        deleted.append(str(p))
    elif p.is_file():
        p.unlink(missing_ok=True)
        deleted.append(str(p))

# 3) Remove Python/Jupyter cache folders
for cache_dir in TRAIN_REPO_DIR.rglob("__pycache__"):
    shutil.rmtree(cache_dir, ignore_errors=True)
for ckpt_dir in TRAIN_REPO_DIR.rglob(".ipynb_checkpoints"):
    shutil.rmtree(ckpt_dir, ignore_errors=True)

# 4) Clear loaded project modules from kernel memory
for mod in list(sys.modules.keys()):
    if mod.startswith("src.") or mod.startswith("src_"):
        del sys.modules[mod]
gc.collect()

print("✅ Fresh start complete")
print(f"Repo: {TRAIN_REPO_DIR}")
print(f"Deleted paths: {len(deleted)}")
for d in deleted:
    print(" -", d)

✅ Fresh start complete
Repo: /content/adaptive_portfolio_rl
Deleted paths: 3
 - /content/adaptive_portfolio_rl/tcn_fusion_results
 - /content/adaptive_portfolio_rl/data/master_features_NORMALIZED.csv
 - /content/adaptive_portfolio_rl/data/daily_ohlcv_assets.csv


In [2]:
#from pathlib import Path
import os

root = Path("/content/adaptive_portfolio_rl")
print("Exists:", root.exists())
print("CWD:", os.getcwd())

print("\nTop-level:")
for p in sorted(root.iterdir()):
    kind = "DIR " if p.is_dir() else "FILE"
    print(f" - [{kind}] {p.name}")

# Quick check for outputs/caches you expected to be deleted
targets = [
    "tcn_fusion_results",
    "tcn_results",
    "tcn_att_results",
    "output_logs",
    "data/phase1_preparation_artifacts",
    "data/master_features_NORMALIZED.csv",
    "data/daily_ohlcv_assets.csv",
    "data/processed_daily_macro_features.csv",
]
print("\nTarget paths:")
for t in targets:
    p = root / t
    print(f" - {t}: {'EXISTS' if p.exists() else 'MISSING'}")


Exists: True
CWD: /content

Top-level:
 - [DIR ] .git
 - [FILE] .gitignore
 - [FILE] RL Portfolio Optimization Feature Engineering.md
 - [FILE] RL_Portfolio_Optimization_Feature_Engineering.ipynb
 - [FILE] USAGE_GUIDE_ACTUARIAL.py
 - [FILE] __init__.py
 - [FILE] convert_md_to_ipynb.py
 - [DIR ] data
 - [FILE] data1.zip
 - [DIR ] data_exports
 - [FILE] debug_attention_weights.py
 - [DIR ] eval
 - [DIR ] paper
 - [FILE] ra_kl_research_writeup.ipynb
 - [FILE] rcdcc_research_writeup.ipynb
 - [FILE] requirements.txt
 - [FILE] run_tcn_eval.py
 - [DIR ] src
 - [FILE] tcn_architecture_analysis.ipynb
 - [DIR ] tcn_documentation
 - [FILE] tcn_evaluation_only.ipynb
 - [FILE] technical_deep_dive_presentation.ipynb
 - [DIR ] tests
 - [FILE] traditional_portfolio_benchmarks.ipynb
 - [DIR ] training_scripts

Target paths:
 - tcn_fusion_results: MISSING
 - tcn_results: MISSING
 - tcn_att_results: MISSING
 - output_logs: MISSING
 - data/phase1_preparation_artifacts: MISSING
 - data/master_features_NORM

In [3]:
#!find /content/adaptive_portfolio_rl -maxdepth 3 | head -n 300

In [4]:
# Install project requirements in Colab VM
#import subprocess, sys
#from pathlib import Path

REPO_DIR = Path("/content/adaptive_portfolio_rl")
REQ_FILE = REPO_DIR / "requirements.txt"

if not REQ_FILE.exists():
    raise FileNotFoundError(f"Missing requirements file: {REQ_FILE}")

print("Using python:", sys.executable)
subprocess.run([sys.executable, "-m", "pip", "install", "--upgrade", "pip", "setuptools", "wheel"], check=True)
subprocess.run([sys.executable, "-m", "pip", "install", "-r", str(REQ_FILE)], check=True)

print("✅ Requirements installed")


Using python: /usr/bin/python3
✅ Requirements installed


In [5]:
# --- GPU sanity/setup for TensorFlow ---
import os
import tensorflow as tf

# 1) Confirm Colab sees an NVIDIA GPU
!nvidia-smi -L

# 2) Confirm TensorFlow sees GPU(s)
gpus = tf.config.list_physical_devices("GPU")
print("TF GPUs:", gpus)
if not gpus:
    raise RuntimeError("No GPU visible to TensorFlow. In Colab: Runtime -> Change runtime type -> GPU")

# 3) Safer GPU memory behavior
for g in gpus:
    tf.config.experimental.set_memory_growth(g, True)

# 4) Optional: speed boost on modern GPUs
tf.keras.mixed_precision.set_global_policy("mixed_float16")
print("Mixed precision policy:", tf.keras.mixed_precision.global_policy())

# 5) Quick proof op runs on GPU
with tf.device("/GPU:0"):
    a = tf.random.normal((4096, 4096))
    b = tf.random.normal((4096, 4096))
    c = tf.matmul(a, b)

print("Matmul device:", c.device)
print("Default GPU device name:", tf.test.gpu_device_name())

GPU 0: NVIDIA A100-SXM4-80GB (UUID: GPU-619e9907-5fe9-d6d9-d317-d9d9b67c559f)
TF GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Mixed precision policy: <DTypePolicy "mixed_float16">
Matmul device: /job:localhost/replica:0/task:0/device:GPU:0
Default GPU device name: /device:GPU:0


In [ ]:
import numpy, pandas, tensorflow
print("numpy", numpy.__version__)
print("pandas", pandas.__version__)
print("tensorflow", tensorflow.__version__)

## 2) Imports

In [6]:
import os, sys
from pathlib import Path

REPO_DIR = Path("/content/adaptive_portfolio_rl")

if not REPO_DIR.exists():
    raise FileNotFoundError(f"Repo not found: {REPO_DIR}")

# Set working directory
os.chdir(REPO_DIR)

# Add repo root to Python path
if str(REPO_DIR) not in sys.path:
    sys.path.insert(0, str(REPO_DIR))

print("cwd:", os.getcwd())
print("sys.path[0]:", sys.path[0])

cwd: /content/adaptive_portfolio_rl
sys.path[0]: /content/adaptive_portfolio_rl


In [7]:
from copy import deepcopy
from pathlib import Path

import pandas as pd

from src.config import get_active_config
from src.csv_logger import CSVLogger
from src.notebook_helpers.tcn_phase1 import prepare_phase1_dataset, run_experiment6_tape

## 3) Base Config and Dataset Prep

In [8]:
# ------------------------------------------------------------------
# Feature lock from CORE project pipeline (actuarial-aware)
# ------------------------------------------------------------------
from src.data_utils import DataProcessor

ACTUARIAL_FEATURE_COLUMNS = [
    "Actuarial_Expected_Recovery",
    "Actuarial_Prob_30d",
    "Actuarial_Prob_60d",
    "Actuarial_Reserve_Severity",
]


def _seed_runtime_feature_families(processor, cfg):
    """
    DataProcessor.get_feature_columns() includes some groups only after runtime build.
    Seed actuarial names from config so pre-build feature locks remain accurate.
    """
    act_enabled = bool(
        cfg.get("feature_params", {}).get("actuarial_params", {}).get("enabled", False)
    )
    if act_enabled:
        processor._actuarial_feature_names = list(ACTUARIAL_FEATURE_COLUMNS)
    return processor


def build_core_active_feature_columns(cfg):
    probe = _seed_runtime_feature_families(DataProcessor(cfg), cfg)
    return list(dict.fromkeys(probe.get_feature_columns("phase1")))


def apply_core_feature_lock(cfg, active_feature_columns):
    # Compute full candidate pool with selection filter temporarily disabled,
    # then enforce active-only by setting disabled_features = full - active.
    # IMPORTANT: preserve preconfigured disabled features (audit drops).
    probe_cfg = deepcopy(cfg)
    probe_fp = probe_cfg.setdefault("feature_params", {})
    probe_fs = probe_fp.setdefault("feature_selection", {})
    probe_fs["disable_features"] = False
    probe_fs["disabled_features"] = []

    probe = _seed_runtime_feature_families(DataProcessor(probe_cfg), probe_cfg)
    core_all_cols = list(dict.fromkeys(probe.get_feature_columns("phase1")))

    active_set = set(active_feature_columns)
    from_core_gap = {c for c in core_all_cols if c not in active_set}

    existing_disabled = set(
        cfg.get("feature_params", {})
        .get("feature_selection", {})
        .get("disabled_features", [])
    )
    disabled = sorted(existing_disabled.union(from_core_gap))

    fp = cfg.setdefault("feature_params", {})
    fs = fp.setdefault("feature_selection", {})
    fs["disable_features"] = True
    fs["disabled_features"] = disabled

    return core_all_cols, disabled


In [9]:
TRAIN_RANDOM_SEED = 42

train_config = deepcopy(get_active_config("phase1"))

# Optional: override analysis horizon
# train_config["ANALYSIS_END_DATE"] = "2025-09-01"

# Build active features from core project files/pipeline.
train_active_feature_columns = build_core_active_feature_columns(train_config)
_, train_disabled_features = apply_core_feature_lock(train_config, train_active_feature_columns)

actuarial_active = [c for c in train_active_feature_columns if c.startswith("Actuarial_")]
print("✅ Core feature lock applied")
print("   active_feature_columns:", len(train_active_feature_columns))
print("   actuarial_active_columns:", len(actuarial_active), actuarial_active)
print("   disabled_features:", len(train_disabled_features))

# Force fresh dataset build and market data re-download
if "train_phase1_data" in globals():
    del train_phase1_data

train_phase1_data = prepare_phase1_dataset(
    train_config,
    force_download=True,
    preparation_artifacts_dir="/content/adaptive_portfolio_rl/data_exports",
)


✅ Core feature lock applied
   active_feature_columns: 50
   disabled_features: 25
📊 Loading raw market data...
   ✅ Raw data shape: (55107, 7)
   ✅ Date range: 2003-09-02 00:00:00 → 2025-08-29 00:00:00

🔧 Computing multi-horizon log returns: [1, 5, 10, 21]
   ✅ Shape after returns: (54897, 11)

📈 Calculating 21-day rolling statistics

🧮 Computing technical indicators

📊 Computing dynamic covariance features

🎯 Adding regime awareness features
   ✅ Master DF shape: (54897, 47)
   ✅ Total features: 47

📊 Integrating fundamental features (if enabled)...
   ✅ Fundamental columns in dataset: 6 (enabled=True)
   🧾 Sample fundamental cols: ['Fundamental_FCFE_Delta', 'Fundamental_Revenue_Delta', 'Fundamental_NCFO_Delta', 'Fundamental_FCFE_Sign', 'Fundamental_Staleness_Days', 'Fundamental_Staleness_Quarters']

📊 Integrating macroeconomic features (if enabled)...
   ✅ Macro features added - 43 columns: ['EFFR_diff', 'EFFR_zscore', 'SOFR_level', 'SOFR_diff', 'FEDFUNDS_diff', 'FEDFUNDS_zscore', '


💾 Saving NORMALISED master dataframe to '/content/adaptive_portfolio_rl/data/master_features_NORMALIZED.csv'

💾 Saved preparation artifacts:
   raw OHLCV: /content/adaptive_portfolio_rl/data_exports/phase1_prep_20260222_222025_raw_ohlcv.csv
   full engineered: /content/adaptive_portfolio_rl/data_exports/phase1_prep_20260222_222025_feature_engineered_full.csv
   analysis-window engineered: /content/adaptive_portfolio_rl/data_exports/phase1_prep_20260222_222025_feature_engineered_analysis_window.csv
   normalized master: /content/adaptive_portfolio_rl/data_exports/phase1_prep_20260222_222025_feature_engineered_normalized.csv
   train normalized: /content/adaptive_portfolio_rl/data_exports/phase1_prep_20260222_222025_train_normalized.csv
   test normalized: /content/adaptive_portfolio_rl/data_exports/phase1_prep_20260222_222025_test_normalized.csv
   scalers: /content/adaptive_portfolio_rl/data_exports/phase1_prep_20260222_222025_scalers.joblib
   audit report: /content/adaptive_portfoli

In [10]:
print("Train shape:", train_phase1_data.train_df.shape)
print("Test shape:", train_phase1_data.test_df.shape)

cols = train_phase1_data.train_df.columns
print("Total columns:", len(cols))

# quick sanity for common redundant groups
dup_like = [c for c in cols if c.endswith("_raw") or c.endswith("_unscaled")]
print("Potential redundant raw/unscaled cols:", len(dup_like))
print(dup_like[:20])

used_now = list(dict.fromkeys(train_phase1_data.data_processor.get_feature_columns("phase1")))
act_now = [c for c in used_now if c.startswith("Actuarial_")]
print("Model feature count (phase1):", len(used_now))
print("Actuarial feature count:", len(act_now), act_now)


Train shape: (43867, 105)
Test shape: (11030, 105)
Total columns: 105
Potential redundant raw/unscaled cols: 0
[]


In [ ]:
cols

In [11]:
used = set(train_phase1_data.data_processor.get_feature_columns("phase1"))
disabled = set(train_config["feature_params"]["feature_selection"]["disabled_features"])
act_used = sorted([c for c in used if c.startswith("Actuarial_")])

print("Used feature count:", len(used))
print("Actuarial used:", len(act_used), act_used)
print("Disabled that still in used:", sorted(disabled & used))  # should be []
print("VIX_zscore used?", "VIX_zscore" in used)


Used feature count: 73
Disabled that still in used: []
VIX_zscore used? False


In [12]:
base_cols = ["Date", "Ticker", "Open", "High", "Low", "Close", "Volume"]
keep = [c for c in base_cols + list(used) if c in train_phase1_data.master_df.columns]

train_phase1_data.master_df = train_phase1_data.master_df[keep].copy()
train_phase1_data.train_df = train_phase1_data.train_df[keep].copy()
train_phase1_data.test_df  = train_phase1_data.test_df[keep].copy()

## 4) Training Overrides (Sharpe-Only Checkpoint Policy)

This policy keeps only Sharpe-threshold high-watermark checkpointing (`>= 0.5`) and disables rare/step/periodic/TAPE checkpoint routes.

In [13]:
# ============================================================================
# NEXT RUN OVERRIDES (post-mortem tuned: KL stability + turnover control)
# ============================================================================
from copy import deepcopy

train_config = deepcopy(train_config)  # or deepcopy(config) if that's your active object

tp = train_config["training_params"]
ap = train_config["agent_params"]
ppo = ap["ppo_params"]
env = train_config["environment_params"]

# ----------------------------------------------------------------------------
# 1) Core run shape
# ----------------------------------------------------------------------------
tp["max_total_timesteps"] = 20_000
tp["timesteps_per_ppo_update"] = 384  # fallback

tp["timesteps_per_ppo_update_schedule"] = [
    {"threshold": 0, "timesteps_per_update": 384},
    {"threshold": 50_000, "timesteps_per_update": 448},
]

tp["batch_size_ppo_schedule"] = [
    {"threshold": 0, "batch_size": 96},
    {"threshold": 50_000, "batch_size": 112},
]

# ----------------------------------------------------------------------------
# 2) PPO stability (reduce aggressiveness)
# ----------------------------------------------------------------------------
ppo["num_ppo_epochs"] = 1
ppo["policy_clip"] = 0.08
ppo["target_kl"] = 0.020
ppo["kl_stop_multiplier"] = 1.25
ppo["minibatches_before_kl_stop"] = 2
ppo["max_grad_norm"] = 0.30

ppo["actor_lr"] = 8e-6
ppo["critic_lr"] = 1.2e-4
ppo["entropy_coef"] = 0.0020

tp["actor_lr_schedule"] = [
    {"threshold": 0, "lr": 8e-6},
    {"threshold": 30_000, "lr": 7e-6},
    {"threshold": 60_000, "lr": 6e-6},
]

# ----------------------------------------------------------------------------
# 3) RA-KL (less aggressive, prevent floor lock)
# ----------------------------------------------------------------------------
tp["ra_kl_enabled"] = True
tp["ra_kl_target_ratio"] = 1.0
tp["ra_kl_ema_alpha"] = 0.25
tp["ra_kl_gain"] = 0.03
tp["ra_kl_deadband"] = 0.20
tp["ra_kl_max_change_fraction"] = 0.05
tp["ra_kl_min_target_kl"] = 0.016
tp["ra_kl_max_target_kl"] = 0.030

# ----------------------------------------------------------------------------
# 4) Dirichlet + concentration controls
# ----------------------------------------------------------------------------
ap["dirichlet_alpha_activation"] = "elu"
ap["dirichlet_logit_temperature"] = 1.15
ap["dirichlet_alpha_cap"] = 35.0
ap["dirichlet_epsilon"] = {"max": 1.0, "min": 0.5}

env["concentration_penalty_scalar"] = 3.0
env["concentration_target_hhi"] = 0.12
env["top_weight_penalty_scalar"] = 2.0
env["action_realization_penalty_scalar"] = 0.5

# ----------------------------------------------------------------------------
# 5) Turnover + execution smoothing
# ----------------------------------------------------------------------------
env["target_turnover"] = 0.35
env["turnover_penalty_scalar"] = 1.5
env["transaction_cost_pct"] = 0.001

tp["action_execution_beta_curriculum"] = {
    0: 0.20,
    30_000: 0.35,
}
tp["evaluation_action_execution_beta"] = 0.35

tp["turnover_penalty_curriculum"] = {
    0: 1.5,
    10_000: 2.0,
    25_000: 2.5,
    40_000: 3.0,
}
tp["evaluation_turnover_penalty_scalar"] = 3.0

# ----------------------------------------------------------------------------
# 6) Episode horizon curriculum (keep cap late)
# ----------------------------------------------------------------------------
tp["use_episode_length_curriculum"] = True
tp["episode_length_curriculum_schedule"] = [
    {"threshold": 0, "limit": 252},
    {"threshold": 10_000, "limit": 504},
    {"threshold": 25_000, "limit": 756},
    {"threshold": 90_000, "limit": 1008},
]

# ----------------------------------------------------------------------------
# 7) Logging + checkpoints
# ----------------------------------------------------------------------------
tp["log_step_diagnostics"] = True
tp["update_log_interval"] = 5
tp["alpha_diversity_log_interval"] = 1
tp["alpha_diversity_warning_after_updates"] = 120
tp["alpha_diversity_warning_std_threshold"] = 0.25

tp["deterministic_validation_checkpointing_enabled"] = True
tp["deterministic_validation_eval_every_episodes"] = 1
tp["deterministic_validation_mode"] = "mean"
tp["deterministic_validation_episode_length_limit"] = None
tp["deterministic_validation_sharpe_min"] = 0.5
tp["deterministic_validation_sharpe_min_delta"] = 0.0
tp["deterministic_validation_seed_offset"] = 10_000
tp["deterministic_validation_checkpointing_only"] = True

tp["high_watermark_checkpoint_enabled"] = False
tp["high_watermark_sharpe_threshold"] = 0.5
tp["step_sharpe_checkpoint_enabled"] = False
tp["periodic_checkpoint_every_steps"] = 0
tp["rare_checkpoint_params"] = {"enable": False}
tp["tape_checkpoint_threshold"] = 999.0

print("✅ Applied next-run override (KL-stable + smoother execution + moderate turnover control)")
print("num_ppo_epochs:", ppo["num_ppo_epochs"])
print("target_kl:", ppo["target_kl"], "| kl_stop_multiplier:", ppo["kl_stop_multiplier"])
print("RA-KL:", {k: tp[k] for k in [
    "ra_kl_enabled", "ra_kl_gain", "ra_kl_deadband",
    "ra_kl_max_change_fraction", "ra_kl_min_target_kl", "ra_kl_max_target_kl"
]})
print("action_execution_beta_curriculum:", tp["action_execution_beta_curriculum"])
print("turnover_penalty_curriculum:", tp["turnover_penalty_curriculum"])
print("concentration:", env["concentration_penalty_scalar"], env["concentration_target_hhi"], env["top_weight_penalty_scalar"])

✅ Applied next-run override (KL-stable + smoother execution + moderate turnover control)
num_ppo_epochs: 1
target_kl: 0.02 | kl_stop_multiplier: 1.25
RA-KL: {'ra_kl_enabled': True, 'ra_kl_gain': 0.03, 'ra_kl_deadband': 0.2, 'ra_kl_max_change_fraction': 0.05, 'ra_kl_min_target_kl': 0.016, 'ra_kl_max_target_kl': 0.03}
action_execution_beta_curriculum: {0: 0.2, 30000: 0.35}
turnover_penalty_curriculum: {0: 1.5, 10000: 2.0, 25000: 2.5, 40000: 3.0}
concentration: 3.0 0.12 2.0


In [15]:
# Optional experimental override (OFF by default)
# Keep this OFF for the aligned default pipeline.
EXPERIMENT_DISABLE_KL_GUARDS = False

if EXPERIMENT_DISABLE_KL_GUARDS:
    tp = train_config["training_params"]
    ppo = train_config["agent_params"]["ppo_params"]

    # Disable RA-KL controller (otherwise it keeps adjusting target_kl)
    tp["ra_kl_enabled"] = False

    # Disable KL early-stop gate in PPOAgentTF
    ppo["target_kl"] = 0.0

    # Optional (irrelevant once target_kl=0, but explicit)
    ppo["kl_stop_multiplier"] = 999.0
    ppo["minibatches_before_kl_stop"] = 9999
    print("⚠️ EXPERIMENT_DISABLE_KL_GUARDS=True (non-default experimental mode)")
else:
    print("ℹ️ EXPERIMENT_DISABLE_KL_GUARDS=False (keeping RA-KL + KL safeguards)")


## 5) Run Training

In [ ]:
RUN_TRAINING = True

if RUN_TRAINING:
    tp = train_config["training_params"]
    print("🚀 Starting training")
    print("Architecture:", train_config["agent_params"].get("actor_critic_type"))
    print("max_total_timesteps:", tp["max_total_timesteps"])

    train_experiment6 = run_experiment6_tape(
        phase1_data=train_phase1_data,
        config=train_config,
        random_seed=TRAIN_RANDOM_SEED,
        csv_logger_cls=CSVLogger,
        use_covariance=True,
        architecture=train_config["agent_params"].get("actor_critic_type"),
        timesteps_per_update=tp.get("timesteps_per_ppo_update", 384),
        max_total_timesteps=tp["max_total_timesteps"],
    )

    print("✅ Training complete")
    print("checkpoint_prefix:", train_experiment6.checkpoint_path)
else:
    print("ℹ️ RUN_TRAINING=False")

🚀 Starting training
Architecture: TCN_FUSION
max_total_timesteps: 20000

EXPERIMENT 6: TCN_FUSION Enhanced + TAPE Three-Component
Architecture: TCN + Fusion
Results root: /content/adaptive_portfolio_rl/tcn_fusion_results
Working dir: /content/adaptive_portfolio_rl
Covariance Features: Yes
🎯 REWARD SYSTEM: TAPE (Three-Component v3)
   Profile: BalancedGrowth
   Daily: Base + DSR/PBRS + Turnover_Proximity
   Terminal: mode=signed | baseline=0.20 | scalar=10.0 (clipped ±10.0)
   Gate A: enabled (Sharpe ≤ 0.00 or MDD ≥ 25.0% -> force non-positive terminal bonus)
   Neutral Band: enabled (±0.020 around baseline)
   🔄 Profile Manager: disabled (static profile only)
🎲 Experiment Seed: 6042 (Base: 42, Offset: 6000)
✅ Features: Enhanced (includes 3 covariance eigenvalues)
   Eigenvalues: ['Covariance_Eigenvalue_0', 'Covariance_Eigenvalue_1', 'Covariance_Eigenvalue_2']
   Train shape: (43867, 80)
   Test shape: (11030, 80)

🏗️ Creating THREE-COMPONENT TAPE v3 environments (with curriculum)...
  

: 

## 6) Inspect Latest Training Logs

In [14]:
TRAIN_RESULTS_ROOT = Path("/content/adaptive_portfolio_rl/tcn_fusion_results")
TRAIN_LOGS_DIR = TRAIN_RESULTS_ROOT / "logs"

episodes_files = sorted(TRAIN_LOGS_DIR.glob("*episodes*.csv"), key=lambda p: p.stat().st_mtime, reverse=True)
if not episodes_files:
    print(f"No episodes CSV found in {TRAIN_LOGS_DIR}")
else:
    train_episodes_path = episodes_files[0]
    train_episodes_df = pd.read_csv(train_episodes_path)
    print("Episodes file:", train_episodes_path)
    print("Rows:", len(train_episodes_df))
    display(train_episodes_df.tail(20))

Episodes file: /content/adaptive_portfolio_rl/tcn_fusion_results/logs/Exp6_TCN_FUSION_Enhanced_TAPE_training_20260222_151442_episodes.csv
Rows: 49


,update,timestep,episode,elapsed_time,episode_return_pct,episode_sharpe,episode_sortino,episode_max_dd,episode_volatility,episode_win_rate,...,actor_grad_norm,critic_grad_norm,alpha_min,alpha_max,alpha_mean,ratio_mean,ratio_std,drawdown_lambda_peak,episode_length,termination_reason
29,150,58816,114,9464.048770,2.190653,0.094299,0.121193,42.686573,0.279999,54.966887,...,1.360187,2.160542,0.883789,4.369141,1.886230,0.955701,0.263988,1.814195,756.0,episode_limit
30,155,61056,117,9820.255982,22.067252,0.439809,0.630596,12.899377,0.123717,52.052980,...,1.189293,1.906437,1.204590,2.972656,1.934082,1.013174,0.230471,0.000000,756.0,episode_limit
31,160,63296,120,10177.620947,41.396616,0.868341,1.116185,15.002422,0.118456,53.642384,...,2.077839,3.396452,1.241536,2.941406,1.929036,1.012059,0.251204,0.000000,756.0,episode_limit
32,165,65536,123,10535.289529,4.296034,0.113843,0.153732,48.160859,0.271998,52.980132,...,1.402580,2.571859,1.263672,3.025391,1.934570,0.990619,0.258082,3.635450,756.0,episode_limit
33,170,67776,126,10895.580393,33.100655,0.591034,0.785112,17.053370,0.146059,52.052980,...,1.389470,1.633517,1.150391,3.085449,1.929932,1.010141,0.228210,0.000553,756.0,episode_limit
34,175,70016,129,11253.437139,20.021064,0.400738,0.562147,13.061167,0.120807,50.331126,...,1.256654,1.593473,1.284180,2.997396,1.935221,0.989271,0.214384,0.000000,756.0,episode_limit
35,180,72256,132,11614.510084,42.966863,0.769392,1.037855,15.876953,0.142571,52.847682,...,1.432416,3.790764,1.226074,3.187500,1.923340,1.015074,0.274607,0.000000,756.0,episode_limit
36,185,74496,135,11971.994982,24.867296,0.518326,0.739074,12.425883,0.118315,50.728477,...,1.983561,1.925111,0.596680,4.523438,1.886230,0.995031,0.477704,0.000000,756.0,episode_limit
37,190,76736,138,12330.298489,1.216329,0.083460,0.114508,46.277679,0.281183,53.774834,...,1.062641,1.497556,1.198730,2.761719,1.960449,0.983898,0.236125,3.409755,756.0,episode_limit
38,195,78976,141,12690.429068,29.862768,0.422073,0.499031,29.803511,0.214142,54.437086,...,1.335551,1.485373,1.184570,2.935547,1.927734,1.008289,0.226884,0.096451,756.0,episode_limit


In [ ]:
train_episodes_df.columns

## 7) Export Results Folder (Optional)
Creates a zip for download from Colab VM.

In [15]:
from pathlib import Path
import subprocess

EXPORT_RESULTS_ZIP = True
EXPORT_PATH = Path("/content/tcn_fusion_results_run3.zip")
ROOT = Path("/content/adaptive_portfolio_rl")

if EXPORT_RESULTS_ZIP:
    # Core items
    include_paths = [
        ROOT / "tcn_fusion_results",
        ROOT / "data" / "phase1_preparation_artifacts",
        ROOT / "data" / "master_features_NORMALIZED.csv",
        ROOT / "data_exports",  # include all prep exports like phase1_prep_* artifacts
    ]

    # Also include latest phase1_prep_* files (explicitly, if present)
    data_exports_dir = ROOT / "data_exports"
    if data_exports_dir.exists():
        latest_prep_files = sorted(
            data_exports_dir.glob("phase1_prep_*"),
            key=lambda p: p.stat().st_mtime,
            reverse=True,
        )
        include_paths.extend(latest_prep_files)

    # De-dup + existence check
    seen = set()
    existing = []
    for p in include_paths:
        p = p.resolve()
        if p.exists() and p not in seen:
            seen.add(p)
            existing.append(p)

    if not existing:
        print("⚠️ Nothing to export.")
    else:
        if EXPORT_PATH.exists():
            EXPORT_PATH.unlink()

        rel_items = [str(p.relative_to(ROOT)) for p in existing if str(p).startswith(str(ROOT))]
        if not rel_items:
            print("⚠️ No export items are under ROOT.")
        else:
            cmd = f"cd {ROOT} && zip -qr {EXPORT_PATH} " + " ".join(f'"{x}"' for x in rel_items)
            subprocess.run(cmd, shell=True, check=True)

            print(f"✅ Created: {EXPORT_PATH}")
            print("Included:")
            for p in rel_items:
                print(" -", p)
else:
    print("ℹ️ EXPORT_RESULTS_ZIP=False")

✅ Created: /content/tcn_fusion_results_run3.zip
Included:
 - tcn_fusion_results
 - data/master_features_NORMALIZED.csv
 - data_exports
 - data_exports/phase1_prep_20260222_151252_preparation_audit.json
 - data_exports/phase1_prep_20260222_151252_scalers.joblib
 - data_exports/phase1_prep_20260222_151252_test_normalized.csv
 - data_exports/phase1_prep_20260222_151252_train_normalized.csv
 - data_exports/phase1_prep_20260222_151252_feature_engineered_normalized.csv
 - data_exports/phase1_prep_20260222_151252_feature_engineered_analysis_window.csv
 - data_exports/phase1_prep_20260222_151252_feature_engineered_full.csv
 - data_exports/phase1_prep_20260222_151252_raw_ohlcv.csv
 - data_exports/phase1_prep_20260222_145210_preparation_audit.json
 - data_exports/phase1_prep_20260222_145210_scalers.joblib
 - data_exports/phase1_prep_20260222_145210_test_normalized.csv
 - data_exports/phase1_prep_20260222_145210_train_normalized.csv
 - data_exports/phase1_prep_20260222_145210_feature_engineered_n

In [16]:
from google.colab import drive
drive.mount('/content/drive')

!cp /content/tcn_fusion_results_run3.zip /content/drive/MyDrive/
print("✅ Copied to Drive: /content/drive/MyDrive/tcn_fusion_results_run3.zip")

Mounted at /content/drive
✅ Copied to Drive: /content/drive/MyDrive/tcn_fusion_results_run3.zip
